In [ ]:
import pandas as pd
import os
import re

# Read the CSV file into a Pandas DataFrame with specified data types
df = pd.read_csv('twcs.csv', dtype={
    'tweet_id': int,
    'author_id': str,
    'inbound': bool,
    'created_at': str,
    'text': str,
    'response_tweet_id': str,
    'in_response_to_tweet_id': str
})

# Display the first 10 rows as a table
display(df.head(10))

In [49]:
# Filter customer care accounts (alphanumeric author_ids)
customer_care_accounts_all = df[pd.to_numeric(df['author_id'], errors='coerce').isna()]['author_id']

# Count tweets for each customer care account
tweet_counts = customer_care_accounts_all.value_counts()

# Display the results in descending order
print(tweet_counts)

author_id
AmazonHelp        169840
AppleSupport      106860
Uber_Support       56270
SpotifyCares       43265
Delta              42253
                   ...  
JackBox              266
OfficeSupport        218
AskDSC               210
CarlsJr              196
HotelTonightCX       152
Name: count, Length: 108, dtype: int64


In [52]:
###########################################################
# This codeblock will split twcs.csv into separate file per 
# customer care account. new field `text_nomentions` has all 
# the twitter mentions removed.
#
# output location:
#        /twcs/by_account/   
###########################################################

# Function to extract tweet IDs from comma-separated strings (handling NaN gracefully)
def extract_tweet_ids(id_str):
    if pd.isna(id_str):
        return []
    elif isinstance(id_str, str):  # Check if it's a string
        return [int(x) for x in id_str.split(',') if x.strip().isdigit()]
    elif isinstance(id_str, list):  # Check if it's already a list
        return id_str  # No need to process further
    else:
        return []  # Handle other unexpected data types

# Apply the function to extract tweet IDs
df['response_tweet_id_list'] = df['response_tweet_id'].apply(extract_tweet_ids)
df['in_response_to_tweet_id_list'] = df['in_response_to_tweet_id'].apply(extract_tweet_ids)

# Identify unique customer care accounts (alphanumeric author_ids)
customer_care_accounts = df[pd.to_numeric(df['author_id'], errors='coerce').isna()]['author_id'].unique()

# Create the output directory if it doesn't exist
output_dir = 'twcs/by_account/'
os.makedirs(output_dir, exist_ok=True)

# Iterate through each customer care account
for account in customer_care_accounts:
    # Filter tweets directly involving the account (author or mentioned in text)
    account_tweets = df[
        (df['author_id'] == account) | 
        (df['text'].str.contains(f"@{account}", case=False, na=False))
    ]

    # Get all tweet IDs related to the account (including responses and in_response_to)
    related_tweet_ids = set(
        account_tweets['tweet_id'].tolist() +
        [item for sublist in account_tweets['response_tweet_id_list'] if isinstance(sublist, list) for item in sublist] + 
        [item for sublist in account_tweets['in_response_to_tweet_id_list'] if isinstance(sublist, list) for item in sublist]
    )

    # Filter all tweets with the related tweet IDs
    all_conversation_tweets = df[df['tweet_id'].isin(related_tweet_ids)].copy()  # Create a copy here

    # Remove @ mentions and trailing spaces from the 'text' field
    all_conversation_tweets['text_nomentions'] = all_conversation_tweets['text'].astype(str).str.replace(r'@[^\s]+', '', regex=True)

    # Reorder columns to place 'text_nomentions' after 'text'
    all_conversation_tweets = all_conversation_tweets[[
        'tweet_id', 'author_id', 'inbound', 'created_at', 'text', 'text_nomentions', 
        'response_tweet_id', 'in_response_to_tweet_id'
    ]]

    # Write the conversation tweets to a CSV file, overwriting if it already exists
    output_file = os.path.join(output_dir, f"{account}.csv")
    all_conversation_tweets.to_csv(output_file, index=False)
    
    # Print the number of rows in the output file
    print(f"{account}.csv: {len(all_conversation_tweets)} rows")

# Print the total number of files generated
print(f"Total files generated: {len(customer_care_accounts)}")    

sprintcare.csv: 48263 rows
Ask_Spectrum.csv: 57474 rows
VerizonSupport.csv: 41472 rows
ChipotleTweets.csv: 41612 rows
AskPlayStation.csv: 42795 rows
marksandspencer.csv: 26061 rows
MicrosoftHelps.csv: 23427 rows
ATVIAssist.csv: 47994 rows
AdobeCare.csv: 20289 rows
AmazonHelp.csv: 367944 rows
XboxSupport.csv: 55383 rows
AirbnbHelp.csv: 19020 rows
nationalrailenq.csv: 10734 rows
AirAsiaSupport.csv: 26593 rows
Morrisons.csv: 22186 rows
NikeSupport.csv: 7709 rows
AskAmex.csv: 20867 rows
McDonalds.csv: 19951 rows
YahooCare.csv: 2145 rows
AskLyft.csv: 26000 rows
UPSHelp.csv: 41470 rows
Delta.csv: 89263 rows
AppleSupport.csv: 235215 rows
Uber_Support.csv: 126019 rows
Tesco.csv: 74350 rows
SpotifyCares.csv: 90459 rows
British_Airways.csv: 60777 rows
comcastcares.csv: 69773 rows
AmericanAir.csv: 87768 rows
TMobileHelp.csv: 74965 rows
VirginTrains.csv: 65789 rows
SouthwestAir.csv: 65514 rows
AskeBay.csv: 20593 rows
hulu_support.csv: 47528 rows
GWRHelp.csv: 46923 rows
sainsburys.csv: 43684 rows
A

In [63]:
# Create a list to store the results
results = []

# Iterate through each customer care account
for account in customer_care_accounts:
    # Filter tweets directly involving the account (author or mentioned in text)
    account_tweets = df[
        (df['author_id'] == account) | 
        (df['text'].str.contains(f"@{account}", case=False, na=False))
    ]

    # Get all tweet IDs related to the account (including responses and in_response_to)
    related_tweet_ids = set(
        account_tweets['tweet_id'].tolist() +
        [item for sublist in account_tweets['response_tweet_id_list'] if isinstance(sublist, list) for item in sublist] + 
        [item for sublist in account_tweets['in_response_to_tweet_id_list'] if isinstance(sublist, list) for item in sublist]
    )

    # Filter all tweets with the related tweet IDs
    all_conversation_tweets = df[df['tweet_id'].isin(related_tweet_ids)].copy()  # Create a copy here
    
    # Count the total number of tweets authored by the account
    total_tweets = all_conversation_tweets[all_conversation_tweets['author_id'] == account].shape[0]
    
    # Count the tweets where the author is the account and "because" is in the text
    because_count = all_conversation_tweets[
        (all_conversation_tweets['author_id'] == account) &
        (all_conversation_tweets['text'].str.contains(r'\bbecause\b', case=False, na=False))
    ].shape[0]
    
    # Calculate the percentage of tweets containing "because"
    if total_tweets > 0:
        because_percentage = (because_count / total_tweets) * 100
    else:
        because_percentage = 0  # Handle cases where there are no tweets for the account

    # Append the results to the list
    results.append((account, because_count, because_percentage))

# Sort the results by because_percentage in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the sorted results
for account, because_count, because_percentage in results:
    print(f"{because_count} tweets by {account} contain the word 'because' ({because_percentage:.2f}% of total tweets)")

205 tweets by ChipotleTweets contain the word 'because' (1.09% of total tweets)
210 tweets by GWRHelp contain the word 'because' (1.08% of total tweets)
445 tweets by SpotifyCares contain the word 'because' (1.03% of total tweets)
13 tweets by Kimpton contain the word 'because' (0.97% of total tweets)
31 tweets by nationalrailenq contain the word 'because' (0.70% of total tweets)
8 tweets by VMUcare contain the word 'because' (0.58% of total tweets)
66 tweets by SW_Help contain the word 'because' (0.54% of total tweets)
45 tweets by McDonalds contain the word 'because' (0.53% of total tweets)
1 tweets by CarlsJr contain the word 'because' (0.51% of total tweets)
22 tweets by VirginAtlantic contain the word 'because' (0.51% of total tweets)
4 tweets by PearsonSupport contain the word 'because' (0.48% of total tweets)
42 tweets by AskeBay contain the word 'because' (0.43% of total tweets)
32 tweets by JetBlue contain the word 'because' (0.40% of total tweets)
10 tweets by Walmart contain

In [62]:
#############################################################
# put questions and answers side-by-side.
# all @ mentions are removed from text as well.
#
# output location:
#        /twcs/by_account2    
#
# schema:
#       in_response_to_tweet_id: this is question from user 
#       text: this response from customer care account
#############################################################


# Apply the function to extract tweet IDs
df['response_tweet_id_list'] = df['response_tweet_id'].apply(extract_tweet_ids)
df['in_response_to_tweet_id_list'] = df['in_response_to_tweet_id'].apply(extract_tweet_ids)

# Convert in_response_to_tweet_id to integer, with NaN replaced by -1 to avoid issues
df['in_response_to_tweet_id'] = pd.to_numeric(df['in_response_to_tweet_id'], errors='coerce').fillna(-1).astype(int)

# Merge the DataFrame to create side-by-side question and answer pairs
merged_df = pd.merge(
    df,
    df[['tweet_id', 'text']].rename(columns={'tweet_id': 'in_response_to_tweet_id', 'text': 'in_response_to_tweet_text'}),
    on='in_response_to_tweet_id',
    how='left'
)

# Remove @ mentions and trailing spaces from both 'in_response_to_tweet_text' and 'text' fields
merged_df['in_response_to_tweet_text'] = merged_df['in_response_to_tweet_text'].astype(str).str.replace(r'@[^\s]+', '', regex=True).str.strip()
merged_df['text'] = merged_df['text'].astype(str).str.replace(r'@[^\s]+', '', regex=True).str.strip()

# Remove records where in_response_to_tweet_id is -1 or in_response_to_tweet_text is NaN
merged_df = merged_df[(merged_df['in_response_to_tweet_id'] != -1) & (merged_df['in_response_to_tweet_text'] != 'nan')]

# Reorder columns for clarity
merged_df = merged_df[[
    'tweet_id', 'author_id', 'inbound', 'created_at', 
    'in_response_to_tweet_id', 'in_response_to_tweet_text', 
    'text', 'response_tweet_id'
]]

# Filter for customer care accounts (alphanumeric author_ids)
customer_care_accounts = merged_df[pd.to_numeric(merged_df['author_id'], errors='coerce').isna()]['author_id'].unique()

# Create the output directory if it doesn't exist
output_dir = 'twcs/by_account2/'
os.makedirs(output_dir, exist_ok=True)

# Iterate through each customer care account
for account in customer_care_accounts:
    # Filter tweets involving the account
    account_tweets = merged_df[
        (merged_df['author_id'] == account)
    ]

    # Write the merged tweets to a CSV file
    output_file = os.path.join(output_dir, f"{account}_merged.csv")
    account_tweets.to_csv(output_file, index=False)
    
    # Print the number of rows in the output file
    print(f"{account}_merged.csv: {len(account_tweets)} rows")

# Print the total number of files generated
print(f"Total files generated: {len(customer_care_accounts)}")

sprintcare_merged.csv: 22313 rows
Ask_Spectrum_merged.csv: 25776 rows
VerizonSupport_merged.csv: 17805 rows
ChipotleTweets_merged.csv: 18599 rows
AskPlayStation_merged.csv: 18675 rows
marksandspencer_merged.csv: 11451 rows
MicrosoftHelps_merged.csv: 11216 rows
ATVIAssist_merged.csv: 17518 rows
AdobeCare_merged.csv: 9876 rows
AmazonHelp_merged.csv: 168822 rows
XboxSupport_merged.csv: 24312 rows
AirbnbHelp_merged.csv: 8821 rows
nationalrailenq_merged.csv: 4135 rows
AirAsiaSupport_merged.csv: 12724 rows
Morrisons_merged.csv: 10021 rows
NikeSupport_merged.csv: 3408 rows
AskAmex_merged.csv: 11093 rows
McDonalds_merged.csv: 8429 rows
YahooCare_merged.csv: 875 rows
AskLyft_merged.csv: 11787 rows
UPSHelp_merged.csv: 17765 rows
Delta_merged.csv: 42149 rows
AppleSupport_merged.csv: 106648 rows
Uber_Support_merged.csv: 56193 rows
Tesco_merged.csv: 38470 rows
SpotifyCares_merged.csv: 43206 rows
British_Airways_merged.csv: 29291 rows
comcastcares_merged.csv: 32975 rows
AmericanAir_merged.csv: 36531